About the Dataset:
The dataset used in this notebook contains hourly energy consumption data from PJM Interconnection LLC (PJM), a regional transmission organization operating in the United States.

In [ ]:
pip install prophet

Importing the Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from prophet import Prophet

import xgboost as xgb
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [ ]:
file_path = "C:/Users/atifi/OneDrive/Desktop/Dataset/pjm_hourly_est.csv"
df = pd.read_csv(file_path)
df = df.set_index('Datetime')
df.index = pd.to_datetime(df.index)

In [ ]:
df.head()

In [ ]:
df.columns

Initial Analysis

In [ ]:
# List of columns to plot
columns_to_plot = ['AEP', 'COMED', 'DAYTON', 'DEOK', 'DOM', 'DUQ', 'EKPC', 'FE', 'NI', 'PJME', 'PJMW', 'PJM_Load']

# Iterate through each column and plot
for column in columns_to_plot:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df[column], mode='lines', name=column))
    fig.update_layout(title=f'{column} Time Series', xaxis_title='Date', yaxis_title='Value')
    fig.show()

In [ ]:
df.info()


In [ ]:
# find the number of missing values in each column in the form of percentage with print statement
print("The number of missing values in each column is :")
df.isnull().sum() / len(df) * 100

In [ ]:
df.describe().T

In [ ]:
# find the number of missing values in each column in the form of percentage with print statement
print("The number of missing values in each column is :")
df.isnull().sum() / len(df) * 100

In [ ]:
# Calculate the percentage of missing values for each column
missing_percentages = (df.isnull().sum() / len(df)) * 100

# Create a bar chart
fig = go.Figure(data=[go.Bar(
    x=missing_percentages.index,
    y=missing_percentages.values,
    marker_color='lightsalmon'  # Adjust the color as needed
)])

# Update layout
fig.update_layout(
    title='Percentage of Missing Values in Each Column',
    xaxis_title='Columns',
    yaxis_title='Percentage Missing (%)'
)

# Show the plot
fig.show()

Observation: We've observed that the PJME column has the fewest missing values compared to other columns in the dataset. Due to its completeness, we've decided to focus our further analysis on this column. This decision ensures that our analysis is based on robust and reliable data, leading to more insightful findings.

Analysis on PMJE

In [ ]:
file_path = "C:/Users/atifi/OneDrive/Desktop/Dataset/PJME_hourly.csv"
pjme = pd.read_csv(file_path)
pjme = pjme.set_index('Datetime')
pjme.index = pd.to_datetime(pjme.index)

In [ ]:
pjme.head()

In [ ]:
from pandas.api.types import CategoricalDtype

cat_type = CategoricalDtype(categories=['Monday','Tuesday',
                                        'Wednesday',
                                        'Thursday','Friday',
                                        'Saturday','Sunday'],
                            ordered=True)

def create_features(df, label=None):
    """
    Creates time series features from datetime index.
    """
    df = df.copy()
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['weekday'] = df['date'].dt.day_name()
    df['weekday'] = df['weekday'].astype(cat_type)
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    
    # Calculate week of the year using isocalendar()
    df['weekofyear'] = df['date'].apply(lambda x: x.isocalendar()[1])
    
    df['date_offset'] = (df.date.dt.month*100 + df.date.dt.day - 320)%1300

    df['season'] = pd.cut(df['date_offset'], [0, 300, 602, 900, 1300], 
                          labels=['Spring', 'Summer', 'Fall', 'Winter']
                   )
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear','weekday',
           'season']]
    if label:
        y = df[label]
        return X, y
    return X

X, y = create_features(pjme, label='PJME_MW')
features_and_target = pd.concat([X, y], axis=1)

In [ ]:
import plotly.express as px

# Drop rows with missing values
features_and_target_cleaned = features_and_target.dropna()

# Create box plot using Plotly Express
fig = px.box(features_and_target_cleaned,
             x='weekday',
             y='PJME_MW',
             color='season',
             title='Power Use MW by Day of Week',
             labels={'weekday': 'Day of Week', 'PJME_MW': 'Energy (MW)'},
             width=800, height=500)

# Show legend outside the plot
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.show()

Train/Test Split of Data

In [ ]:
# Split the data into training and test sets
split_date = '2015-01-01'
pjme_train = pjme.loc[pjme.index <= split_date].copy()
pjme_test = pjme.loc[pjme.index > split_date].copy()

# Rename columns for clarity
pjme_train['Set'] = 'Training Set'
pjme_test['Set'] = 'Test Set'

# Combine training and test sets
combined_data = pd.concat([pjme_train, pjme_test])

# Create a scatter plot using Plotly Express
fig = px.scatter(combined_data, x=combined_data.index, y='PJME_MW', color='Set',
                 title='PJM East: Training and Test Sets',
                 labels={'x': 'Date', 'y': 'Energy (MW)'},
                 width=1000, height=600)

# Update plot aesthetics
fig.update_traces(marker=dict(size=3, opacity=0.7))

# Show legend outside the plot
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.show()

Prophet Model¶
The Prophet model is a powerful tool for time series forecasting developed by Facebook. It is designed to handle various time series data, including those with daily, weekly, or yearly seasonality, as well as holiday effects.

When using the Prophet model, it expects the dataset to be structured in a specific way. The key requirements are:

Datetime Column: The dataset should contain a datetime column representing the timestamps of the observations. This column is typically named ds, short for datetime stamp.

Target Column: The Prophet model requires a target variable that it aims to forecast. This column, representing the values we want to predict, should be named y.

By adhering to these naming conventions, we ensure compatibility with the Prophet model and facilitate seamless integration into our forecasting workflow.

In [ ]:
import time

df_train_prophet = pjme_train.reset_index() \
    .rename(columns={'Datetime':'ds',
                     'PJME_MW':'y'})

start_time = time.time()

model = Prophet()
model.fit(df_train_prophet)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

In [ ]:
df_test_prophet = pjme_test.reset_index() \
    .rename(columns={'Datetime':'ds',
                     'PJME_MW':'y'})

df_test_fcst = model.predict(df_test_prophet)

In [ ]:
import plotly.graph_objects as go

# Convert the forecast DataFrame to Plotly-compatible format
forecast_plotly = df_test_fcst.reset_index()

# Create the figure
fig = go.Figure()

# Add the actual data as a scatter plot
fig.add_trace(go.Scatter(x=pjme_test.index, y=pjme_test['PJME_MW'], mode='markers', name='Actual', marker=dict(size=5)))

# Add the forecasted values and confidence intervals as line plots
fig.add_trace(go.Scatter(x=forecast_plotly['ds'], y=forecast_plotly['yhat'], mode='lines', name='Forecast'))
fig.add_trace(go.Scatter(x=forecast_plotly['ds'], y=forecast_plotly['yhat_lower'], mode='lines', name='Lower Bound', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=forecast_plotly['ds'], y=forecast_plotly['yhat_upper'], mode='lines', name='Upper Bound', line=dict(dash='dash')))

# Update layout
fig.update_layout(title='Prophet Forecast',
                  xaxis_title='Date',
                  yaxis_title='Energy Consumption (MW)',
                  width=1000, height=600)

# Show the plot
fig.show()

In [ ]:
fig = model.plot_components(df_test_fcst)
plt.show()

Forecast to Actual Comparing

In [ ]:
import matplotlib.pyplot as plt

# Plot actuals
plt.figure(figsize=(15, 5))  # Set desired figure size
plt.scatter(pjme_test.index, pjme_test["PJME_MW"], color='blue', label='Actuals')

# Plot forecast using Prophet's built-in plotting
fig = model.plot(df_test_fcst)
plt.legend()  # Add legend after Prophet's plot (optional)

# Customize labels and title (optional)
plt.xlabel('Date')
plt.ylabel('PJME MW')
plt.title('PJME MW Forecast and Actuals')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming pjme_test has a datetime index and pjme_test['PJME_MW'] is the data
# to be plotted

# Create figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# Plot actuals
ax.scatter(pjme_test.index, pjme_test["PJME_MW"], color='blue', label='Actuals')

# Plot forecast using Prophet's built-in plotting
model.plot(df_test_fcst, ax=ax)

# Convert date strings to datetime objects
lower_bound = pd.to_datetime('01-01-2015')
upper_bound = pd.to_datetime('02-01-2015')

# Set x-axis bounds using datetime objects
ax.set_xbound(lower_bound, upper_bound)

# Set y-axis limits (optional)
ax.set_ylim(0, 60000)  # Adjust as needed

# Add title and legend (optional)
plt.suptitle('January 2015 Forecast vs Actuals')
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd  # Import pandas for date handling

# Assuming pjme_test has a datetime index and pjme_test['PJME_MW'] is the data
# to be plotted

# Create figure and axes
fig, ax = plt.subplots(figsize=(15, 5))

# Plot actuals
ax.scatter(pjme_test.index, pjme_test["PJME_MW"], color='blue', label='Actuals')

# Plot forecast using Prophet's built-in plotting
model.plot(df_test_fcst, ax=ax)

# Convert date strings to datetime objects for x-axis bounds
lower_bound = pd.to_datetime('01-01-2015')
upper_bound = pd.to_datetime('01-08-2015')

# Set x-axis bounds using datetime objects
ax.set_xbound(lower_bound, upper_bound)

# Set y-axis limits (optional)
ax.set_ylim(0, 60000)  # Adjust as needed

# Add title
ax.set_title('First Week of January Forecast vs Actuals')

# Add legend (optional)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

Model Evaluation with Error Matrix

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_true=pjme_test['PJME_MW'], y_pred=df_test_fcst['yhat']))

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_true=pjme_test['PJME_MW'], y_pred=df_test_fcst['yhat'])

# Print the results
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")